In [1]:
import sklearn
import pandas as pd

import logging

import torch
cuda_available = torch.cuda.is_available()
import wandb
from simpletransformers.ner import NERModel, NERArgs

d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [2]:
train_df=pd.read_csv("..\..\Dataset\\bert\\train.csv")
eval_df=pd.read_csv("..\..\Dataset\\bert\\val.csv")


In [3]:
train_df

,sentence_id,words,labels
0,1382601382042103808,Hidup,O
1,1382601382042103808,sesedih,O
2,1382601382042103808,dan,O
3,1382601382042103808,secaper,O
4,1382601382042103808,apa,O
...,...,...,...
108840,1383795163412058114,Gubernur,O
108841,1383795163412058114,Khofifah,B-PER
108842,1383795163412058114,fokus,O
108843,1383795163412058114,untuk,O


In [4]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [5]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    # "num_train_epochs": 1,
    "num_train_epochs": 5,
    "weight_decay": 0,
    # "learning_rate": 4e-5,
    "learning_rate": 0.00006828,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,
}


In [11]:
sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [2, 3, 5]},
        "learning_rate": {"min": 5e-5, "max": 4e-4},
    },
}

In [12]:
sweep_id = wandb.sweep(sweep_config, project="ner sweep")

Create sweep with ID: dpnikvql
Sweep URL: https://wandb.ai/anwarfarihin/ner%20sweep/sweeps/dpnikvql


In [13]:
def train():
    wandb.init()
    model = NERModel('bert', 
                    'indolem/indobert-base-uncased',
                    args=model_args,
                    labels=label_list,
                    use_cuda=cuda_available,
                    sweep_config=wandb.config
                    )
    model.train_model(train_df, eval_df=eval_df)
    model.eval_model(eval_df)
    wandb.join()

In [24]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: fev3o68z with config:
wandb: 	learning_rate: 0.0014197744241588898
wandb: 	num_train_epochs: 5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anwarfarihin (use `wandb login --relogin` to force relogin)


Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at indolem/indobert-base-uncas

Run fev3o68z errored: RuntimeError('CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 2.94 GiB already allocated; 0 bytes free; 2.99 GiB reserved in total by PyTorch)')
wandb: ERROR Run fev3o68z errored: RuntimeError('CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 2.94 GiB already allocated; 0 bytes free; 2.99 GiB reserved in total by PyTorch)')
wandb: Agent Starting Run: hp4i7bvf with config:
wandb: 	learning_rate: 0.00150582522846751
wandb: 	num_train_epochs: 3
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at indolem/indobert-base-uncas

Run hp4i7bvf errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 4.00 GiB total capacity; 2.94 GiB already allocated; 0 bytes free; 2.99 GiB reserved in total by PyTorch)')
wandb: ERROR Run hp4i7bvf errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 4.00 GiB total capacity; 2.94 GiB already allocated; 0 bytes free; 2.99 GiB reserved in total by PyTorch)')
wandb: Agent Starting Run: c3dz6l1s with config:
wandb: 	learning_rate: 0.0038366880173895204
wandb: 	num_train_epochs: 2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at indolem/indobert-base-uncas

Run c3dz6l1s errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 4.00 GiB total capacity; 2.94 GiB already allocated; 0 bytes free; 2.99 GiB reserved in total by PyTorch)')
wandb: ERROR Run c3dz6l1s errored: RuntimeError('CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 4.00 GiB total capacity; 2.94 GiB already allocated; 0 bytes free; 2.99 GiB reserved in total by PyTorch)')
Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: ERROR Detected 3 failed runs in the first 60 seconds, killing sweep.
wandb: To disable this check set WANDB_AGENT_DISABLE_FLAPPING=true
